In [ ]:
#hide
#default_exp doclinks

# nbdev.doclinks
- Generating a documentation index from a module

In [ ]:
#export
from nbdev.read import *
from nbdev.export import *
from nbdev.imports import *
from fastcore.script import *
from fastcore.imports import *

import ast,contextlib
from pprint import pformat

In [ ]:
from fastcore.test import *
from pdb import set_trace
from importlib import reload

In [ ]:
everything_fn = '../tests/01_everything.ipynb'
ExportModuleProcessor('../tests/00_some.thing.ipynb', 'tmp').create_modules()
proc = ExportModuleProcessor(everything_fn, 'tmp')
proc.create_modules()

In [ ]:
#export
def mod_fn2name(fn):
    "Convert filename `fn` to its module name"
    return '.'.join(str(Path(fn).with_suffix('')).split('/'))

In [ ]:
mod_fn = Path('tmp/everything.py')
test_eq(mod_fn2name(mod_fn), 'tmp.everything')

In [ ]:
#export
class DocLinks:
    def __init__(self, mod_fn, doc_func, dest_fn, mod_name=None):
        mod_fn,dest_fn = Path(mod_fn),Path(dest_fn)
        if mod_name is None: mod_name = mod_fn2name(
            mod_fn.resolve().relative_to(dest_fn.parent.parent.resolve()))
        store_attr()

In [ ]:
def _help(s): return f"help for {s}"

dest_fn = Path('tmp/_nbdev.py')
link = DocLinks(mod_fn, _help, dest_fn)
link.mod_name

'tmp.everything'

In [ ]:
#export
@patch
def create_nbdev_idx_tmpl(self:DocLinks):
    "Create basic template for an nbdev documentation index file"
    if not self.dest_fn.exists():
        self.dest_fn.write_text("# Autogenerated by nbdev\n\nmods = {}\n\nsyms = {}")

In [ ]:
with contextlib.suppress(FileNotFoundError): dest_fn.unlink()

link.create_nbdev_idx_tmpl()
assert "Autogenerated" in dest_fn.read_text()

In [ ]:
#export
@patch
def _update_mods(self:DocLinks, mods):
    mods[self.mod_name] = self.doc_func(self.mod_name)
    return pformat(mods, width=160)

@patch
def update_mods(self:DocLinks):
    "Update `mods` definition in `dest_fn` to add `mod_name` linked to `doc_func()`"
    update_var('mods', self._update_mods, self.dest_fn)

In [ ]:
link.update_mods()

import tmp._nbdev
reload(tmp._nbdev)
mod_name = mod_fn2name(mod_fn)
test_eq(tmp._nbdev.mods, {mod_name: _help(mod_name)})

In [ ]:
#export
def _all_or_exports(fn):
    code = Path(fn).read_text()
    trees = L(ast.parse(code).body)
    res = read_var(code, '__all__')
    return L(retr_exports(trees) if res is None else res),trees

def _is_patch(o): return any(L(o.decorator_list).filter(Self.id.startswith('patch')))
def _pat_name(o):
    try: return f'{o.args.args[0].annotation.id}.{o.name}'
    except AttributeError: return None 

def _exp_meths(tree):
    return L(f"{tree.name}.{o.name}" for o in tree.body
             if isinstance(o,(ast.FunctionDef,ast.AsyncFunctionDef)) and o.name[0]!='_')

In [ ]:
#export
@patch
def _update_syms(self:DocLinks, syms):
    exp,trees = _all_or_exports(self.mod_fn)
    exp_class = trees.filter(lambda o: isinstance(o, ast.ClassDef) and o.name in exp)
    exp += exp_class.map(_exp_meths).concat()
    pats = L(f'{o.args.args[0].annotation.id}.{o.name}' for o in trees
       if isinstance(o,(ast.FunctionDef,ast.AsyncFunctionDef)) and _is_patch(o))
    exp += pats.filter()
    exp = exp.map(f"{self.mod_name}.{{}}")
    syms[self.mod_name] = exp.map_dict(self.doc_func)
    return pformat(syms, width=160)

@patch
def update_syms(self:DocLinks):
    "Update `mods` definition in `dest_fn` to add `mod_name` linked to `doc_func()`"
    update_var('syms', self._update_syms, self.dest_fn)

In [ ]:
symn = 'tmp.everything.a_y'
link.update_syms()
import tmp._nbdev
reload(tmp._nbdev)
test_eq(tmp._nbdev.syms[mod_name][symn], _help(symn))
test_eq(set(tmp._nbdev.syms[mod_name].keys()),
        set(L('m_y', 'n_y', 'q_y', 'a_y', 'b_y', 'd_y', 'e_y', 'o_y', 'p_y', 'd_y.di_n', 'd_y.d3i_n'
             ).map('tmp.everything.{}')))

In [ ]:
#export
@patch
def build_index(self:DocLinks):
    self.create_nbdev_idx_tmpl()
    self.update_mods()
    self.update_syms()

## CLI

In [ ]:
#export
def _update_baseurl(path=None):
    "Add or update `baseurl` in `_config.yml` for the docs"
    _re_baseurl = re.compile('^baseurl\s*:.*$', re.MULTILINE)
    path = Path(ifnone(path, Config().doc_path))
    fname = path/'_config.yml'
    if not fname.exists(): return
    code = fname.read_text()
    if _re_baseurl.search(code) is None: code = code + f"\nbaseurl: {Config().doc_baseurl}"
    else: code = _re_baseurl.sub(f"baseurl: {Config().doc_baseurl}", code)
    fname.write_text(code)

def _use_nb(p): return not p.name.startswith('_') and '.ipynb_checkpoints' not in p.parts

In [ ]:
#export
@call_parse
def nbdev_build_lib(
    nbs:Param("Glob specifiying notebooks to export (defaults to all nbs in `nbs_path`)", str)=None,
    dest:Param("Destination for library (defaults to `lib_path`)", str)=None):
    "Convert notebooks matching `nbs` to modules"
    cfg = Config()
    dest = cfg.config_path/(ifnone(dest, cfg.lib_path))
    if os.environ.get('IN_TEST',0): return
    _nbdev = dest/'_nbdev.py'
    if nbs is None:
        files = L(cfg.path('nbs_path').glob('*.ipynb')).filter(_use_nb)
        with contextlib.suppress(FileNotFoundError): _nbdev.unlink()
    else: files = glob.glob(nbs)
    for file in files: ExportModuleProcessor(file, dest).create_modules()
    for file in dest.glob("**/*.py"):
        if file.name[0]!='_': DocLinks(file, _help, dest/'_nbdev.py').build_index()
    if not cfg.get('extension',False):
        _update_baseurl()
        add_init(dest)

In [ ]:
nbdev_build_lib()

In [ ]:
print(Path('../nbdev/_nbdev.py').read_text())

# Autogenerated by nbdev

mods = {'nbdev.cli': 'help for nbdev.cli',
 'nbdev.doclinks': 'help for nbdev.doclinks',
 'nbdev.export': 'help for nbdev.export',
 'nbdev.imports': 'help for nbdev.imports',
 'nbdev.read': 'help for nbdev.read',
 'nbdev.showdoc': 'help for nbdev.showdoc',
 'nbdev.sync': 'help for nbdev.sync',
 'nbdev.tutorial': 'help for nbdev.tutorial',
 'nbdev.version': 'help for nbdev.version'}

syms = {'nbdev.cli': {'nbdev.cli.nbdev_diff_nbs': 'help for nbdev.cli.nbdev_diff_nbs', 'nbdev.cli.nbdev_trust_nbs': 'help for nbdev.cli.nbdev_trust_nbs'},
 'nbdev.doclinks': {'nbdev.doclinks.DocLinks': 'help for nbdev.doclinks.DocLinks',
                    'nbdev.doclinks.DocLinks._update_mods': 'help for nbdev.doclinks.DocLinks._update_mods',
                    'nbdev.doclinks.DocLinks._update_syms': 'help for nbdev.doclinks.DocLinks._update_syms',
                    'nbdev.doclinks.DocLinks.build_index': 'help for nbdev.doclinks.DocLinks.build_index',
                    'nbde

In [ ]:
from nbdev import __version__
from nbdev.read import *
import nbdev.export
reload(nbdev.export)
assert __version__
assert hasattr(nbdev.export, 'read_nb')